# 1. Data Preprocessing & Parquet
- Process the CSV file "marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv"
- Cleanup & Create two Parquet files:
  - `products.parquet` (containing product `id`, `title`, `brand`, `category`, `price`, `rating`, `features`, and `ingredients`)

## Load CSV Data

- Load the 'marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv' file into a pandas DataFrame.
- Inspect the column names to understand their structure before proceeding with extraction.

Please Ensure to use your own path to the csv file when running the file

In [ ]:
import pandas as pd

# Modify to your path to CSV
file_path = '/content/drive/MyDrive/genAI final - personal/marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv'

df = pd.read_csv(file_path)

print("First 5 rows of the DataFrame:")
print(df.head(3))
print("\nColumn names in the DataFrame:")
print(df.columns.tolist())

### Check for null columns

In [ ]:
print(df.info())

## Filter to Toys/Games/Party/Crafts domain slice

Our team has decided to use Toys/Games/Party/Crafts domain slice, which is a dominant slice in the dataset (>8000 rows)
- Filter based on toy keywords to achieve the slice

In [ ]:
# ============================================================
# Filter to a Toys/Games/Party/Crafts domain slice
#    (based on Category text keywords)
# ============================================================

toy_keywords = [
    "Toys", "Toy", "Game", "Games", "Puzzle", "Jigsaw", "Figure", "Figures",
    "Doll", "Dolls", "Party", "Stuffed", "Plush", "Costume", "Craft", "Crafts",
    "Kids", "Play", "Vehicle", "Learning"
]

pattern = "|".join(toy_keywords)

mask_toys = df["Category"].str.contains(pattern, case=False, na=False)
toys_df = df[mask_toys].copy()

print("Toys/Party/Craft slice size:", len(toys_df))

## Prepare Products Data

- Extract, Clean-up, and transform the relevant columns from the loaded CSV data to create a DataFrame for `products.parquet`.


In [ ]:
import numpy as np

# Initialize products_df
products_df = pd.DataFrame({
    'id': toys_df['Uniq Id'],
    'title': toys_df['Product Name'],
    'brand': toys_df['Brand Name'],
    'category': toys_df['Category']
})

# Clean and convert 'Selling Price' to numeric for 'price' column
products_df['price'] = toys_df['Selling Price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
products_df['price'] = pd.to_numeric(products_df['price'], errors='coerce')

# Initialize 'rating' column with NaN
products_df['rating'] = np.nan
# Fill the rating with random rating (numeric between 1-5)
products_df['rating'] = np.random.randint(1, 6, size=len(products_df))

# Concatenate text from relevant columns (not null) for 'features'
feature_columns = [
    'About Product',
    'Product Specification',
    'Technical Details'
]

# Fill NaN values with empty strings before concatenation
products_df['features'] = toys_df[feature_columns].fillna('').agg(' '.join, axis=1)

# Populate 'ingredients' column (but Ingredients column is all null)
products_df['ingredients'] = toys_df['Ingredients']

# Display the first few rows of products_df
print("\nFirst 5 rows of products_df:")
print(products_df.head())

# Check data types and non-null counts for products_df
print("\nInfo for products_df:")
products_df.info()

In [ ]:
products_df.to_csv('products_df.csv', index=False)

## Fill in Ingredients using Meta-Llama-3-8B-Instruct (keyword extraction from features column)

Since ingredients are NaN in all rows, we will extract ingredients from features if available. If not, column value is filled as "Unknown"

Here is one example:

In [ ]:
products_df.iloc[1]['features']

"Plastic is made of corn and are 100% compostable"
We are expecting Llama to extract "corn, plastic" in this case.


In [ ]:
!pip install transformers accelerate sentencepiece --quiet


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

In [ ]:
sample_df = products_df.copy()

In [ ]:

# Build context for each row
def build_product_context(row):
    parts = []
    for col in ["title", "features"]:
        if col in sample_df.columns:
            val = row.get(col)
        else:
            val = None
        if isinstance(val, str) and val.strip():
            parts.append(val.strip())
    return "\n".join(parts)


# LLM inference with HF Transformers

def generate_ingredient_keywords(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return "unknown"

    system_prompt = (
        "You are an e-commerce data annotator.\n"
        "Given the product name and description, infer the likely main ingredients "
        "or material-related keywords of the product.\n"
        "- Answer ONLY in concise English.\n"
        "- Output a comma-separated list (e.g., 'aloe vera, glycerin, water').\n"
        "- If it is not a consumable, cosmetic, household chemical, pet/baby product, "
        "or you cannot infer ingredients, respond with 'unknown'."
    )

    user_prompt = f"Product information:\n{text}\n\nInferred ingredients or material keywords:"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # LLaMA-3 chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=64,
            temperature=0.1,
            do_sample=False,
        )

    # get just the response
    gen_tokens = output[0][inputs["input_ids"].shape[1]:]
    answer = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

    # just the first row
    answer = answer.split("\n")[0].strip()

    lower = answer.lower()
    if "unknown" in lower or "not possible to infer" in lower:
        return "unknown"

    # Clean up patterns like "The answer is: polyester"
    for sep in ["the answer is", "answer:", "answer is"]:
        if sep in lower:
            idx = lower.find(sep)
            answer = answer[idx + len(sep):].strip(" :.").strip()
            break

    return answer


# Filter target rows with features
mask = sample_df["features"].notna()
target_df = sample_df[mask]

results = []

print(f"Target rows: {len(target_df)}")


# Run inference
# Start with 20 rows for prompt tuning
for i, (idx, row) in enumerate(target_df.iterrows(), start=1):
    ctx = build_product_context(row)
    ing_keywords = generate_ingredient_keywords(ctx)
    results.append((idx, ing_keywords))

    if i % 20 == 0:  # progress indicator
        print(f"Processed {i} rows.")


# Fill ingredients column
for idx, ing_keywords in results:
    sample_df.loc[idx, "ingredients"] = ing_keywords


# Save result
sample_df.to_csv("products_with_ingredients_generated.csv", index=False)
print("Saved: products_with_ingredients_generated.csv")

In [ ]:
import pandas as pd

sample_df = pd.read_csv('products_with_ingredients_generated.csv')

Check that ingredients field is correctly populated

In [ ]:
sample_df.loc[0:5, 'ingredients']

We also see "corn, plastic" correctly populated in the example we mentioned above.

## Check the info on clean up df again



In [ ]:
sample_df.info()

## Fill in the "Brands" Column using Llama

In [ ]:
# Build context for each row
def build_product_context(row):
    parts = []
    for col in ["title", "features"]:
        if col in sample_df.columns:
            val = row.get(col)
        else:
            val = None
        if isinstance(val, str) and val.strip():
            parts.append(val.strip())
    return "\n".join(parts)


# LLM inference with HF Transformers

def generate_brand_name(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return "unknown"

    system_prompt = (
        "You are an e-commerce data annotator.\n"
        "Given the product name and description, infer the most likely BRAND NAME "
        "or MANUFACTURER NAME of the product.\n"
        "- Answer ONLY with the brand/manufacturer name in concise English.\n"
        "- Do NOT include any extra words such as 'Brand:' or full sentences.\n"
        "- If you cannot infer a clear brand or manufacturer, respond with 'unknown'."
    )

    user_prompt = f"Product information:\n{text}\n\nInferred brand or manufacturer name:"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # LLaMA-3 chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=32,
            temperature=0.1,
            do_sample=False,
        )

    # get just the response
    gen_tokens = output[0][inputs["input_ids"].shape[1]:]
    answer = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

    # just the first line
    answer = answer.split("\n")[0].strip()

    lower = answer.lower()
    if "unknown" in lower or "not possible to infer" in lower:
        return "unknown"

    # Clean up patterns like "The answer is: Disney"
    for sep in ["the answer is", "answer:", "answer is", "brand:", "brand is"]:
        if sep in lower:
            idx = lower.find(sep)
            answer = answer[idx + len(sep):].strip(" :.").strip()
            break

    return answer


# Filter target rows with features / title / description
mask = (
    sample_df.get("title", "").notna()
    | sample_df.get("features", "").notna()
)
target_df = sample_df[mask]

results = []

print(f"Target rows: {len(target_df)}")


# Run inference
# Start with 20 rows for prompt tuning
for i, (idx, row) in enumerate(target_df.iterrows(), start=1):
    ctx = build_product_context(row)
    brand_name = generate_brand_name(ctx)
    results.append((idx, brand_name))

    if i % 20 == 0:  # progress indicator
        print(f"Processed {i} rows.")


# Fill brand_generated column
if "brand" not in sample_df.columns:
    sample_df["brand"] = None

for idx, brand_name in results:
    sample_df.loc[idx, "brand"] = brand_name


# Save result
sample_df.to_csv("products_with_brand_generated.csv", index=False)
print("Saved: products_with_brand_generated.csv")

In [ ]:
print("First 5 rows of products_df:")
print(sample_df['brand'].head(5))

## Clean-up "Features"

In [ ]:
sample_df['features'].head(5)

In [ ]:
import re

def clean_features(text: str) -> str:
    if not isinstance(text, str):
        return text

    # 1. Remove the repeated default Amazon phrase
    prefix = "Make sure this fits by entering your model number. | "
    if text.startswith(prefix):
        text = text[len(prefix):]

    # 2. Remove emojis / non-ASCII characters
    # Keep only ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # 3. Strip whitespace
    text = text.strip()

    # 4. Remove double spaces
    text = re.sub(r"\s+", " ", text)

    return text if text else None


# Apply to the whole column
sample_df["features"] = sample_df["features"].apply(clean_features)

print("Done cleaning!")
sample_df["features"].head()

## Merge Rating fetched from Google Shopping API
Our group was able to fetch first 1200 rows' rating from the API, but was limited due to API # of request restriction on the free account.

We will merge these 1200 results, but randomly generate the rating for the rest of the products

In [ ]:
rating_df = pd.read_csv('/content/Amazon_Product_Dataset_with_Rating.csv')

In [ ]:
rating_df.head(5)

In [ ]:
# print number of nan in rating_df["rating"]
print(rating_df["rating"].isna().sum())

In [ ]:
import numpy as np

# 1) Merge rating_df ratings into cleaned_df
merged = sample_df.merge(
    rating_df[["id", "rating"]],
    on="id",
    how="left",
    suffixes=("", "_real")
)

# 2) Fill NaN rating with random numbers
missing_mask = merged["rating_real"].isna()
merged.loc[missing_mask, "rating_real"] = np.round(np.random.uniform(1.0, 5.0, size=missing_mask.sum()), 1)

# 3) Ensure final rating is float
merged["rating"] = merged["rating_real"].astype(float)

# Drop helper column
merged = merged.drop(columns=["rating_real"])

merged.head()

## Final Review of the Cleaned Data and Export as CSV

In [ ]:
merged.head(5)

In [ ]:
merged.to_csv('products_df_merged_cleaned.csv', index=False)

## Save Products Data to Parquet

Save the prepared products DataFrame to a Parquet file named `products.parquet`.


In [ ]:
merged.to_parquet('products.parquet', index=False)
print("merged successfully saved to 'products.parquet'")